In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
# dataset.py

import os
import argparse
import time
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from scipy import sparse

class DataLoader():
    '''
    Load BOJ dataset
    '''
    def __init__(self, path):        
        self.pro_dir = os.path.join(path, 'pro_sg')
        print(self.pro_dir)
        assert os.path.exists(self.pro_dir), "Preprocessed files do not exist. Run data.py"

        self.n_items = self.load_n_items()
    
    def load_data(self, datatype='train'):
        if datatype == 'train':
            return self._load_train_data()
        elif datatype == 'validation':
            return self._load_tr_te_data(datatype)
        elif datatype == 'test':
            return self._load_tr_te_data(datatype)
        else:
            raise ValueError("datatype should be in [train, validation, test]")
        
    def load_n_items(self):
        '''
        아이템의 개수를 반환하는 함수.
        '''
        unique_sid = list()
        with open(os.path.join(self.pro_dir, 'unique_sid.txt'), 'r') as f:
            for line in f:
                unique_sid.append(line.strip())
        n_items = len(unique_sid)
        return n_items
    
    def _load_train_data(self):
        '''
        train 데이터를 csr_matrix로 반환해주는 함수
        '''
        path = os.path.join(self.pro_dir, 'train.csv')
        
        tp = pd.read_csv(path)
        n_users = tp['uid'].max() + 1

        rows, cols = tp['uid'], tp['sid']
        data = sparse.csr_matrix((np.ones_like(rows),
                                 (rows, cols)), dtype='float64',
                                 shape=(n_users, self.n_items))
        return data
    
    def _load_tr_te_data(self, datatype='test'):
        tr_path = os.path.join(self.pro_dir, '{}_tr.csv'.format(datatype))
        te_path = os.path.join(self.pro_dir, '{}_te.csv'.format(datatype))

        tp_tr = pd.read_csv(tr_path)
        tp_te = pd.read_csv(te_path)

        start_idx = min(tp_tr['uid'].min(), tp_te['uid'].min())
        end_idx = max(tp_tr['uid'].max(), tp_te['uid'].max())
        
        # print('start_idx :', start_idx)
        # print('end_idx :', end_idx)

        rows_tr, cols_tr = tp_tr['uid'] - start_idx, tp_tr['sid']
        rows_te, cols_te = tp_te['uid'] - start_idx, tp_te['sid']

        data_tr = sparse.csr_matrix((np.ones_like(rows_tr),
                                    (rows_tr, cols_tr)), dtype='float64', shape=(end_idx - start_idx + 1, self.n_items))
        data_te = sparse.csr_matrix((np.ones_like(rows_te),
                                    (rows_te, cols_te)), dtype='float64', shape=(end_idx - start_idx + 1, self.n_items))
        return data_tr, data_te

In [5]:
pip install bottleneck

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 355 kB 3.0 MB/s 


In [6]:
# utils.py

import os
from scipy import sparse
import time
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os
import bottleneck as bn
import numpy as np
import time
from copy import deepcopy
from math import log, floor, ceil


def load_train_data(csv_file, n_items, n_users, global_indexing=False):
    tp = pd.read_csv(csv_file)
    
    n_users = n_users if global_indexing else tp['uid'].max() + 1

    rows, cols = tp['uid'], tp['sid']
    data = sparse.csr_matrix((np.ones_like(rows),
                             (rows, cols)), dtype='float64',
                             shape=(n_users, n_items))
    return data


def load_tr_te_data(csv_file_tr, csv_file_te, n_items, n_users, global_indexing=False):
    tp_tr = pd.read_csv(csv_file_tr)
    tp_te = pd.read_csv(csv_file_te)

    if global_indexing:
        start_idx = 0
        end_idx = len(unique_uid) - 1
    else:
        start_idx = min(tp_tr['uid'].min(), tp_te['uid'].min())
        end_idx = max(tp_tr['uid'].max(), tp_te['uid'].max())

    rows_tr, cols_tr = tp_tr['uid'] - start_idx, tp_tr['sid']
    rows_te, cols_te = tp_te['uid'] - start_idx, tp_te['sid']

    data_tr = sparse.csr_matrix((np.ones_like(rows_tr),
                             (rows_tr, cols_tr)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    data_te = sparse.csr_matrix((np.ones_like(rows_te),
                             (rows_te, cols_te)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    return data_tr, data_te


def get_data(dataset, global_indexing=False):
    unique_sid = list()
    unique_uid = list()
    with open(os.path.join(dataset, 'unique_sid.txt'), 'r') as f:
        for line in f:
            unique_sid.append(line.strip())

    with open(os.path.join(dataset, 'unique_uid.txt'), 'r') as f:
        for line in f:
            unique_uid.append(line.strip())
    
    # unique_uid = list()
    # with open(os.path.join(dataset, 'unique_uid.txt'), 'r') as f:
    #     for line in f:
    #         unique_uid.append(line.strip())
            
    n_items = len(unique_sid)
    n_users = len(unique_uid)
    
    train_data = load_train_data(os.path.join(dataset, 'train.csv'), n_items, n_users, global_indexing=global_indexing)


    vad_data_tr, vad_data_te = load_tr_te_data(os.path.join(dataset, 'validation_tr.csv'),
                                               os.path.join(dataset, 'validation_te.csv'),
                                               n_items, n_users, 
                                               global_indexing=global_indexing)

    test_data_tr, test_data_te = load_tr_te_data(os.path.join(dataset, 'test_tr.csv'),
                                                 os.path.join(dataset, 'test_te.csv'),
                                                 n_items, n_users, 
                                                 global_indexing=global_indexing)
    
    data = train_data, vad_data_tr, vad_data_te, test_data_tr, test_data_te
    data = (x.astype('float32') for x in data)
    
    return data




def get_count(df, id):
    '''
    df -> DataFrame
    id -> Feature of DataFrame
    '''
    interaction_count_groupby_id = df[[id]].groupby(id, as_index=False)
    grouped_count = interaction_count_groupby_id.size()
    return grouped_count

# 특정한 횟수 이상의 리뷰가 존재하는(사용자의 경우 min_uc 이상, 아이템의 경우 min_sc이상) 
# 데이터만을 추출할 때 사용하는 함수입니다.
# 현재 데이터셋에서는 결과적으로 원본그대로 사용하게 됩니다.
def filter_triplets(df, min_user_interaction, min_problem_interaction):
    user_interaction_count = get_count(df, 'user')
    problem_interaction_count = get_count(df, 'item')

    print(f"Size of Dataframe Before Filtering: {df.size}")

    if min_user_interaction > 0:
        df = df[df['user'].isin(user_interaction_count[user_interaction_count['size'] >= min_user_interaction]['user'])]

    print(f"Size of Dataframe After User Filtering: {df.size}")

    if min_problem_interaction > 0:
        df = df[df['item'].isin(problem_interaction_count[problem_interaction_count['size'] >= min_problem_interaction]['item'])]

    print(f"Size of Dataframe After Problem Filtering: {df.size}")

    return df, user_interaction_count, problem_interaction_count

#훈련된 모델을 이용해 검증할 데이터를 분리하는 함수입니다.
#100개의 액션이 있다면, 그중에 test_prop 비율 만큼을 비워두고, 그것을 모델이 예측할 수 있는지를
#확인하기 위함입니다.
def split_train_test_proportion(data, test_prop=0.2):
    '''
    data -> DataFrame
    
    train과 test를 8:2 비율로 나눠주는 함수.
    '''
    data_grouped_by_user = data.groupby('user')
    tr_list, te_list = list(), list()

    np.random.seed(98765)
    
    for _, group in data_grouped_by_user:
        n_items_u = len(group)
        
        if n_items_u >= 5:
            idx = np.zeros(n_items_u, dtype='bool') # 'False'가 n_items_u개 만큼 채워진 array
            
            # n_items_u개 중에서 20%의 인덱스를 랜덤으로 뽑아서 해당 인덱스를 'True'로 바꿈
            idx[np.random.choice(n_items_u, size=int(test_prop * n_items_u), replace=False).astype('int64')] = True
                    
            tr_list.append(group[np.logical_not(idx)]) # 'False'인 것을 tr_list에 추가
            te_list.append(group[idx]) # 'True'인 것을 te_list에 추가
        
        else:
            tr_list.append(group)
    
    data_tr = pd.concat(tr_list)
    data_te = pd.concat(te_list)

    return data_tr, data_te

def numerize(tp, profile2id, show2id):
    '''
    tp -> DataFrame
    user2id, item2id -> dict()
    
    user, item을 reindexing한 df 반환.
    '''
    uid = tp['user'].apply(lambda x: profile2id[x])
    sid = tp['item'].apply(lambda x: show2id[x])
    return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])


def sparse2torch_sparse(data):
    """
    Convert scipy sparse matrix to torch sparse tensor with L2 Normalization
    This is much faster than naive use of torch.FloatTensor(data.toarray())
    https://discuss.pytorch.org/t/sparse-tensor-use-cases/22047/2
    """
    samples = data.shape[0]
    features = data.shape[1]
    coo_data = data.tocoo()
    indices = torch.LongTensor([coo_data.row, coo_data.col])
    row_norms_inv = 1 / np.sqrt(data.sum(1))
    row2val = {i : row_norms_inv[i].item() for i in range(samples)}
    values = np.array([row2val[r] for r in coo_data.row])
    t = torch.sparse.FloatTensor(indices, torch.from_numpy(values).float(), [samples, features])
    return t

def naive_sparse2tensor(data):
    return torch.FloatTensor(data.toarray())


def numerize_for_infer(tp, profile2id, show2id):
    uid = tp['user'].apply(lambda x: profile2id[str(x)])
    sid = tp['item'].apply(lambda x: show2id[str(x)])
    return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])

def de_numerize(tp, re_p2id, re_s2id):
    uid2 = tp['user'].apply(lambda x: re_p2id[x])
    sid2 = tp['item'].apply(lambda x: re_s2id[x])
    return pd.DataFrame(data={'uid': uid2, 'sid': sid2}, columns=['uid', 'sid'])

def ndcg(X_pred, heldout_batch, k=100):
    '''
    Normalized Discounted Cumulative Gain@k for binary relevance
    ASSUMPTIONS: all the 0's in heldout_data indicate 0 relevance
    '''
    batch_users = X_pred.shape[0]
    idx_topk_part = bn.argpartition(-X_pred, k, axis=1)
    topk_part = X_pred[np.arange(batch_users)[:, np.newaxis],
                       idx_topk_part[:, :k]]
    idx_part = np.argsort(-topk_part, axis=1)

    idx_topk = idx_topk_part[np.arange(batch_users)[:, np.newaxis], idx_part]

    tp = 1. / np.log2(np.arange(2, k + 2))

    e = 0.000001

    DCG = (heldout_batch[np.arange(batch_users)[:, np.newaxis],
                         idx_topk].toarray() * tp).sum(axis=1)
    IDCG = np.array([(tp[:min(n, k)]).sum()
                     for n in heldout_batch.getnnz(axis=1)]) + e
    return DCG / IDCG


def recall(X_pred, heldout_batch, k=100):
    batch_users = X_pred.shape[0]

    idx = bn.argpartition(-X_pred, k, axis=1)
    X_pred_binary = np.zeros_like(X_pred, dtype=bool)
    X_pred_binary[np.arange(batch_users)[:, np.newaxis], idx[:, :k]] = True

    e = 0.000001

    X_true_binary = (heldout_batch > 0).toarray()
    tmp = (np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)).astype(
        np.float32)
    #-------------------------
    # print('tmp :', len(tmp))
    # print('np.minimum(k, X_true_binary.sum(axis=1) :',len(np.minimum(k, X_true_binary.sum(axis=1))))
    #-------------------------
    recall = tmp / np.minimum(k, X_true_binary.sum(axis=1) + e)
    return recall

def min_max(lv):
    if lv <= 5:
        return 0, 7
    elif lv <= 10:
        return 5, 12
    elif lv <= 13:
        return 8, 16
    elif lv <= 15:
        return 11, 18
    else:
        return (floor(lv-log(lv, 2)), ceil(lv+log(lv, 3)))

def infer(user, item, dict_user_lv, dict_item_lv, id2show, infer_cnt):
    pred = np.array([])
    user_lv = dict_user_lv[user]
    cnt = 0
    mini, maxi = min_max(user_lv)
    item = sorted(item, reverse=True)
    for i in item:
        item_lv = dict_item_lv[int(id2show[i])]
        if mini <= item_lv <= maxi:
            pred = np.append(pred, i)
            cnt += 1
    
        if cnt == infer_cnt:
            return pred
    else:
        # print('else user :', user)
        if len(pred) < infer_cnt:
            for i in item:
                if i not in pred:
                    pred = np.append(pred, i)

                if len(pred) == infer_cnt:
                    return np.array(pred)

In [10]:
# config.py

# ====================================================
# CFG
# ====================================================
class args:
    dataset = '/home/recognizer14/airflow/dags/module_models/rec_general'
    hidden_dim = 600
    latent_dim = 200
    batch_size = 500
    beta = None
    gamma = 0.005
    lr = 5e-4
    n_epochs = 10
    n_enc_epochs = 3
    n_dec_epochs = 1
    not_alternating = False
    data = 'data/train/'
    cuda = False
    seed = 1111
    early_stopping = 5
    infer_cnt = 20
    save = 'model.pt'
    total_anneal_steps = 200000
    anneal_cap = 0.2
    log_dir = './logs'
    wd = 0.00
    log_interval = 100
    save_dir = '/home/recognizer14/airflow/dags/module_models/rec_general/best_models'

In [11]:
# preprocessing.py

import time
import torch
import numpy as np
#from .dataset import *
#from .utils import *
import json
import pandas as pd
from torch import nn

import os
import warnings
warnings.filterwarnings(action='ignore')

## 각종 파라미터 세팅
#from .config import *

# Set the random seed manually for reproductibility.
torch.manual_seed(args.seed)

def preprocessing_all(raw_data, df_problems, db):
    #만약 GPU가 사용가능한 환경이라면 GPU를 사용
    if torch.cuda.is_available():
        args.cuda = True

    device = torch.device("cuda" if args.cuda else "cpu")

    print("Load and Preprocess BOJ dataset")
    # Load Data
    # Filter Data
#    raw_data, user_activity, item_popularity = filter_triplets(raw_data, 5, 10)
    #---------------------------
    # 여기에 tag == None인 아이템은 제외
#    df_problems = pd.read_sql('select * from problems', db)
#    df_problems.drop(df_problems[df_problems.average_tries == 7340].index, axis=0, inplace=True)
    # level 0에 해당하는 문제 제거
#    df_problems = df_problems[df_problems.level != 0]
    # not_solvable == False만
#    df_problems = df_problems[df_problems.is_solvable == True]
    # tag가 nan인 문제 제거
#    df_problems = df_problems[~df_problems.tags.isnull()]

 #   raw_data = raw_data[raw_data['item'].isin(df_problems['problem_id'].values)].reset_index(drop=True)
    #---------------------------


    # Shuffle User Indices
    unique_uid = pd.unique(raw_data['user'])
    print("len(unique_uid): ", len(unique_uid))
    print("(BEFORE) unique_uid:",unique_uid)
    np.random.seed(98765)
    idx_perm = np.random.permutation(unique_uid.size) # 해당 숫자까지의 인덱스를 무작위로 섞은 것을 arr로 반환
    unique_uid = unique_uid[idx_perm]
    print("(AFTER) unique_uid:",unique_uid) # 무작위로 item을 섞음

    n_users = unique_uid.size
    n_heldout_users = 3000

    # Split Train/Validation/Test User Indices
    tr_users = unique_uid[:(n_users - n_heldout_users * 2)]
    vd_users = unique_uid[(n_users - n_heldout_users * 2): (n_users - n_heldout_users)]
    te_users = unique_uid[(n_users - n_heldout_users):]

    #주의: 데이터의 수가 아닌 사용자의 수입니다!
    print("훈련 데이터에 사용될 사용자 수:", len(tr_users))
    print("검증 데이터에 사용될 사용자 수:", len(vd_users))
    print("테스트 데이터에 사용될 사용자 수:", len(te_users))

    ##훈련 데이터에 해당하는 아이템들
    #Train에는 전체 데이터를 사용합니다.
    train_plays = raw_data.loc[raw_data['user'].isin(tr_users)]

    ##아이템 ID
    unique_sid = pd.unique(raw_data['item'])
    pro_dir = os.path.join(args.dataset, 'pro_sg')

    if not os.path.exists(pro_dir):
        os.makedirs(pro_dir)

    item2id = dict((int(sid), int(i)) for (i, sid) in enumerate(unique_sid)) # item2idx dict
    user2id = dict((pid, int(i)) for (i, pid) in enumerate(unique_uid)) # user2idx dict

    with open(os.path.join(pro_dir, 'item2id.json'), 'w', encoding="utf-8") as f:
        json.dump(item2id, f, ensure_ascii=False, indent="\t")
        
    with open(os.path.join(pro_dir, 'user2id.json'), 'w', encoding="utf-8") as f:
        json.dump(user2id, f, ensure_ascii=False, indent="\t")

    with open(os.path.join(pro_dir, 'unique_sid.txt'), 'w') as f:
        for sid in unique_sid:
            f.write('%s\n' % sid)

    with open(os.path.join(pro_dir, 'unique_uid.txt'), 'w') as f:
        for uid in unique_uid:
            f.write('%s\n' % uid)

    #Validation과 Test에는 input으로 사용될 tr 데이터와 정답을 확인하기 위한 te 데이터로 분리되었습니다.
    print('Data Split Start!')
    vad_plays = raw_data.loc[raw_data['user'].isin(vd_users)]
    vad_plays = vad_plays.loc[vad_plays['item'].isin(unique_sid)]
    vad_plays_tr, vad_plays_te = split_train_test_proportion(vad_plays)

    test_plays = raw_data.loc[raw_data['user'].isin(te_users)]
    test_plays = test_plays.loc[test_plays['item'].isin(unique_sid)]
    test_plays_tr, test_plays_te = split_train_test_proportion(test_plays)

    train_data = numerize(train_plays, user2id, item2id)
    train_data.to_csv(os.path.join(pro_dir, 'train.csv'), index=False)

    vad_data_tr = numerize(vad_plays_tr, user2id, item2id)
    vad_data_tr.to_csv(os.path.join(pro_dir, 'validation_tr.csv'), index=False)

    vad_data_te = numerize(vad_plays_te, user2id, item2id)
    vad_data_te.to_csv(os.path.join(pro_dir, 'validation_te.csv'), index=False)

    test_data_tr = numerize(test_plays_tr, user2id, item2id)
    test_data_tr.to_csv(os.path.join(pro_dir, 'test_tr.csv'), index=False)

    test_data_te = numerize(test_plays_te, user2id, item2id)
    test_data_te.to_csv(os.path.join(pro_dir, 'test_te.csv'), index=False)
    print("Data Split Done!")

    # item_tag_emb
    print("Item tag emb start!")
    set_tags = set()
    for tags in df_problems['tags'].dropna().values:
        for tag in tags.split(','):
            set_tags.add(tag)


    df_tags = df_problems[['problem_id', 'tags']]
    df_tags['tags'] = df_tags['tags'].str.split(',')
    # df_tags = df_tags.explode('tags').dropna().reset_index(drop=True)
    df_tags = df_tags.explode('tags').dropna().reset_index(drop=True)
    df_tags = df_tags[df_tags['problem_id'].isin(unique_sid)].reset_index(drop=True)

    temp = nn.Embedding(len(set_tags), 300)
    tag_emb = pd.DataFrame(df_tags['tags'].value_counts().index.values, columns=['tags'])

    dict_tag_idx = dict()
    for i, j in enumerate(df_tags['tags'].value_counts().index.values):
        dict_tag_idx[j] = i

    list_emb = []
    dict_tag_emb = dict()
    for i in df_tags['tags'].value_counts().index.values:
        list_emb.append(temp(torch.tensor(dict_tag_idx[i])).detach().numpy())
        dict_tag_emb[i] = temp(torch.tensor(dict_tag_idx[i])).detach().numpy()

    df_tag_emb = pd.concat([tag_emb, pd.DataFrame(list_emb)], axis=1)
    df_tags2 = pd.merge(df_tags, df_tag_emb, on='tags', how='left')
    tag2emb = df_tags2.iloc[:, 2:].values
    df_tags['emb'] = list(tag2emb) 

    total = []
    def item_genre_emb_mean(i):
        total.append(np.mean(df_tags[df_tags['problem_id'] == i].emb))

    item_genre_emb_idx = pd.DataFrame(list(df_tags['problem_id'].unique()), columns=['item'])
    item_genre_emb_idx.item.apply(lambda x: item_genre_emb_mean(x))
    item_genre_emb = pd.DataFrame(total)
    item_genre_emb.index = df_tags['problem_id'].unique()

    item_genre_emb = item_genre_emb.reset_index()
    item_genre_emb['index'] = item_genre_emb['index'].apply(lambda x : item2id[x])
    item_genre_emb = item_genre_emb.set_index('index')
    item_genre_emb = item_genre_emb.sort_index()

    item_genre_emb = item_genre_emb.T
    print(item_genre_emb.shape)

    item_genre_emb.to_csv(pro_dir + '/item_tag_emb.csv', index=False)
    print('Item tag emb Done!')

    model_score = dict()
    model_score['recvae'] = 0
    model_score['vae'] = 0
    model_score['dae'] = 0
    with open(os.path.join(pro_dir, 'model_score.json'), 'w', encoding="utf-8") as f:
        json.dump(model_score, f, ensure_ascii=False, indent="\t")
    #--------------------------------------------

In [12]:
# models.py

import numpy as np
from copy import deepcopy

import torch
from torch import nn
from torch.nn import functional as F

# RecVAE
# ------------------------------------------------------------------
def swish(x):
    return x.mul(torch.sigmoid(x))

def log_norm_pdf(x, mu, logvar):
    return -0.5*(logvar + np.log(2 * np.pi) + (x - mu).pow(2) / logvar.exp())


class CompositePrior(nn.Module):
    def __init__(self, hidden_dim, latent_dim, input_dim, mixture_weights=[3/20, 3/4, 1/10]):
        super(CompositePrior, self).__init__()

        self.mixture_weights = mixture_weights

        self.mu_prior = nn.Parameter(torch.Tensor(1, latent_dim), requires_grad=False)
        self.mu_prior.data.fill_(0)

        self.logvar_prior = nn.Parameter(torch.Tensor(1, latent_dim), requires_grad=False)
        self.logvar_prior.data.fill_(0)

        self.logvar_uniform_prior = nn.Parameter(torch.Tensor(1, latent_dim), requires_grad=False)
        self.logvar_uniform_prior.data.fill_(10)

        self.encoder_old = Encoder(hidden_dim, latent_dim, input_dim)
        self.encoder_old.requires_grad_(False)

    def forward(self, x, z):
        post_mu, post_logvar = self.encoder_old(x, 0)

        stnd_prior = log_norm_pdf(z, self.mu_prior, self.logvar_prior)
        post_prior = log_norm_pdf(z, post_mu, post_logvar)
        unif_prior = log_norm_pdf(z, self.mu_prior, self.logvar_uniform_prior)

        gaussians = [stnd_prior, post_prior, unif_prior]
        gaussians = [g.add(np.log(w)) for g, w in zip(gaussians, self.mixture_weights)]

        density_per_gaussian = torch.stack(gaussians, dim=-1)

        return torch.logsumexp(density_per_gaussian, dim=-1)


class Encoder(nn.Module):
    def __init__(self, hidden_dim, latent_dim, input_dim, eps=1e-1):
        super(Encoder, self).__init__()

        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.ln1 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.ln2 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.ln3 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc4 = nn.Linear(hidden_dim, hidden_dim)
        self.ln4 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc5 = nn.Linear(hidden_dim, hidden_dim)
        self.ln5 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)

    def forward(self, x, dropout_rate):
        norm = x.pow(2).sum(dim=-1).sqrt()
        x = x / norm[:, None]

        x = F.dropout(x, p=dropout_rate, training=self.training)

        h1 = self.ln1(swish(self.fc1(x)))
        h2 = self.ln2(swish(self.fc2(h1) + h1))
        h3 = self.ln3(swish(self.fc3(h2) + h1 + h2))
        h4 = self.ln4(swish(self.fc4(h3) + h1 + h2 + h3))
        h5 = self.ln5(swish(self.fc5(h4) + h1 + h2 + h3 + h4))
        return self.fc_mu(h5), self.fc_logvar(h5)


class RecVAE(nn.Module):
    def __init__(self, hidden_dim, latent_dim, input_dim):
        super(RecVAE, self).__init__()

        self.encoder = Encoder(hidden_dim, latent_dim, input_dim)
        self.prior = CompositePrior(hidden_dim, latent_dim, input_dim)
        self.decoder = nn.Linear(latent_dim, input_dim)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, user_ratings, beta=None, gamma=1, dropout_rate=0.5, calculate_loss=True):
        mu, logvar = self.encoder(user_ratings, dropout_rate=dropout_rate)
        z = self.reparameterize(mu, logvar)
        x_pred = self.decoder(z)

        if calculate_loss:
            if gamma:
                norm = user_ratings.sum(dim=-1)
                kl_weight = gamma * norm
            elif beta:
                kl_weight = beta

            mll = (F.log_softmax(x_pred, dim=-1) * user_ratings).sum(dim=-1).mean()
            kld = (log_norm_pdf(z, mu, logvar) - self.prior(user_ratings, z)).sum(dim=-1).mul(kl_weight).mean()
            negative_elbo = -(mll - kld)

            return (mll, kld), negative_elbo

        else:
            return x_pred

    def update_prior(self):
        self.prior.encoder_old.load_state_dict(deepcopy(self.encoder.state_dict()))

# Multi-VAE
# ------------------------------------------------------------------
class MultiVAE(nn.Module):
    """
    Container module for Multi-VAE
    Multi-VAE : Variational Autoencoder with Multinomial Likelihood
    See Variational Autoencoders for Collaborative Filtering
    https://arxiv.org/abs/1802.05814
    """

    def __init__(self, p_dims, q_dims=None, tag_emb=None, title_emb=None, dropout=0.5):
        super(MultiVAE, self).__init__()

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.item_tag = torch.Tensor(tag_emb.values).to(self.device)

        self.p_dims = p_dims
        if q_dims:
            assert q_dims[0] == p_dims[-1], "In and Out dimensions must equal to each other"
            assert q_dims[-1] == p_dims[0], "Latent dimension for p- and q- network mismatches."
            self.q_dims = q_dims
        else:
            self.q_dims = p_dims[::-1]

        # Last dimension of q- network is for mean and variance
        temp_q_dims = self.q_dims[:-1] + [self.q_dims[-1] * 2]

        self.q_layers = nn.ModuleList([nn.Linear(d_in, d_out) for
                                       d_in, d_out in zip(temp_q_dims[:-1], temp_q_dims[1:])])

        self.p_layers = nn.ModuleList([nn.Linear(d_in, d_out) for
                                       d_in, d_out in zip(self.p_dims[:-1], self.p_dims[1:])])

        self.drop = nn.Dropout(dropout)
        self.init_weights()

    def forward(self, input):
        self.input = input

        mu, logvar = self.encode(self.input)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

    def encode(self, input):
        h = F.normalize(input)
        h = self.drop(h)
        h = h.to(self.device)
        h = torch.cat((self.item_tag, h), 0)

        mu, logvar = None, None
        for i, layer in enumerate(self.q_layers):
            h = layer(h)
            if i != len(self.q_layers) - 1:
                h = F.tanh(h)
            else:
                mu = h[:, :self.q_dims[-1]]
                logvar = h[:, self.q_dims[-1]:]
        return mu, logvar

    def reparameterize(self, mu, logvar):
        if self.training:  # 이게 뭐지..?
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h = z
        for i, layer in enumerate(self.p_layers):
            h = layer(h)
            if i != len(self.p_layers) - 1:
                h = F.tanh(h)

        item_tag_emb, reconstructed_h = h.split(
            [self.item_tag.shape[0], self.input.shape[0]], 0)
        return reconstructed_h

    def init_weights(self):
        for layer in self.q_layers:
            # Xavier Initialization for weights
            size = layer.weight.size()
            fan_out = size[0]
            fan_in = size[1]
            std = np.sqrt(2.0 / (fan_in + fan_out))
            layer.weight.data.normal_(0.0, std)

            # Normal Initialization for Biases
            layer.bias.data.normal_(0.0, 0.001)

        for layer in self.p_layers:
            # Xavier Initialization for weights
            size = layer.weight.size()
            fan_out = size[0]
            fan_in = size[1]
            std = np.sqrt(2.0 / (fan_in + fan_out))
            layer.weight.data.normal_(0.0, std)

            # Normal Initialization for Biases
            layer.bias.data.normal_(0.0, 0.001)



def loss_function_vae(recon_x, x, mu, logvar, anneal=1.0):
    BCE = -torch.mean(torch.sum(F.log_softmax(recon_x, 1) * x, -1))
    KLD = -0.5 * torch.mean(torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1))

    return BCE + anneal * KLD

# Multi-DAE
# ------------------------------------------------------------------
class MultiDAE(nn.Module):
    """
    Container module for Multi-DAE
    Multi-DAE : Denoising Autoencoder with Multinomial Likelihood
    See Variational Autoencoders for Collaborative Filtering
    https://arxiv.org/abs/1802.05814
    """

    def __init__(self, p_dims, q_dims=None, tag_emb=None, dropout=0.5):
        super(MultiDAE, self).__init__()

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.item_tag = torch.Tensor(tag_emb.values).to(self.device)
        self.p_dims = p_dims
        if q_dims:
            assert q_dims[0] == p_dims[-1], "In and Out dimensions must equal to each other"
            assert q_dims[-1] == p_dims[0], "Latent dimension for p- and q- network mismatches."
            self.q_dims = q_dims
        else:
            self.q_dims = p_dims[::-1]  # 리스트를 역으로 뒤집음

        self.dims = self.q_dims + self.p_dims[1:]
        self.layers = nn.ModuleList([nn.Linear(d_in, d_out) for
                                     d_in, d_out in zip(self.dims[:-1], self.dims[1:])])
        self.drop = nn.Dropout(dropout)

        self.init_weights()

    def forward(self, input):
        h = F.normalize(input)
        h = self.drop(h)
        h = h.to(self.device)
        h = torch.cat((self.item_tag, h), 0)

        for i, layer in enumerate(self.layers):
            h = layer(h)
            if i != len(self.layers) - 1:
                h = F.tanh(h)

        item_tag_emb, reconstructed_h = h.split([self.item_tag.shape[0], input.shape[0]], 0)
        return reconstructed_h

    def init_weights(self):
        for layer in self.layers:
            # Xavier Initialization for weights
            size = layer.weight.size()
            fan_out = size[0]
            fan_in = size[1]
            std = np.sqrt(2.0 / (fan_in + fan_out))
            layer.weight.data.normal_(0.0, std)

            # Normal Initialization for Biases
            layer.bias.data.normal_(0.0, 0.001)



def loss_function_dae(recon_x, x):
    BCE = -torch.mean(torch.sum(F.log_softmax(recon_x, 1) * x, -1))
    return BCE

In [13]:
# inference.py

import time
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from scipy import sparse
import os
#from .dataset import *
#from .utils import *
import json
from tqdm import tqdm
#from .models import *
import datetime
import warnings
warnings.filterwarnings(action = 'ignore')

import bottleneck as bn

## 각종 파라미터 세팅
#from .config import *

seed = args.seed
np.random.seed(seed)
torch.manual_seed(seed)

def infer_all(raw_data, df_problems, db):
    ## 배치사이즈 포함
    ### 데이터 준비
    # Load Data
    pro_dir = os.path.join(args.dataset, 'pro_sg')

    with open(pro_dir + '/model_score.json', 'r', encoding="utf-8") as f:
        model_score = json.load(f)

#    raw_data, _, _ = filter_triplets(raw_data, 5, 10)
#    df_problems = pd.read_sql('select * from problems', db)
#    df_problems.drop(df_problems[df_problems.average_tries == 7340].index, axis=0, inplace=True)
#    # level 0에 해당하는 문제 제거
#    df_problems = df_problems[df_problems.level != 0]
#    # not_solvable == False만
#    df_problems = df_problems[df_problems.is_solvable == True]
#    # tag가 nan인 문제 제거
#    df_problems = df_problems[~df_problems.tags.isnull()]
#
#    raw_data = raw_data[raw_data['item'].isin(df_problems['problem_id'].values)].reset_index(drop=True)
    print("raw_data nunique:", raw_data.user.nunique())

    if torch.cuda.is_available():
        args.cuda = True
#    device = torch.device("cuda")
    device = torch.device("cuda" if args.cuda else "cpu")
    #device = "cpu"

    # Import Data
    print("Inference Start!!")

    with open(pro_dir + '/item2id.json', 'r', encoding="utf-8") as f:
        show2id = json.load(f)

    with open(pro_dir + '/user2id.json', 'r', encoding="utf-8") as f:
        profile2id = json.load(f)

    infer_df = numerize_for_infer(raw_data, profile2id, show2id)

    loader = DataLoader(args.dataset)
    n_items = loader.load_n_items()
    n_users = infer_df['uid'].max() + 1

    rows, cols = infer_df['uid'], infer_df['sid']
    data = sparse.csr_matrix((np.ones_like(rows),(rows, cols)), dtype='float64', shape=(n_users, n_items))

    num_data = data.shape[0]
    index_list = list(range(num_data))

    df_users = pd.read_sql('select * from users', db)
    df_users = df_users[['handle', 'tier']] # base_dir 경로를 수정해야 합니다.
    df_user_lv = df_users[df_users['handle'].isin(raw_data['user'].unique())].reset_index(drop=True)

    dict_user_lv = dict()
    for i in range(len(df_user_lv)):
        dict_user_lv[df_user_lv.iloc[i,0]] = df_user_lv.iloc[i, 1]

    dict_item_lv = dict()
    for i in range(len(df_problems)):
        dict_item_lv[df_problems.iloc[i, 1]] = df_problems.iloc[i,6]

    id2profile = dict((int(v), k) for k, v in profile2id.items())
    id2show = dict((int(v), k) for k, v in show2id.items())

    model_kwargs = {
    'hidden_dim': args.hidden_dim,
    'latent_dim': args.latent_dim,
    'input_dim': data.shape[1]
    }

    model_name = ""
    score = 0
    for m, s in model_score.items():
        if s > score:
            print("확인")
            model_name = m
            score = s
        print(m, s)
        print(score)
    print(f"Best Model => {model_name} : {score:.4f}")

    if model_name == 'recvae':        
        model = RecVAE(**model_kwargs)
        model.load_state_dict(torch.load(args.save_dir + '/best_recvae_' + args.save))
        model = model.to(device)
    
    elif model_name == 'vae':
        p_dims = [200, 3000, n_items] # [200, 600, 6807]
        item_tag_emb = pd.read_csv(pro_dir + '/item_tag_emb.csv')

        model = MultiVAE(p_dims, tag_emb=item_tag_emb)

        with open(os.path.join(args.save_dir, 'best_vae_' + args.save), 'rb') as f:
            model.load_state_dict(torch.load(f))
        model = model.to(device)

    elif model_name == 'dae':
        p_dims = [200, 3000, n_items]  # [200, 600, 6807]
        item_tag_emb = pd.read_csv(pro_dir + '/item_tag_emb.csv')

        model = MultiDAE(p_dims, tag_emb=item_tag_emb).to(device)

        with open(os.path.join(args.save_dir, 'best_dae_' + args.save), 'rb') as f:
            model.load_state_dict(torch.load(f))
        model = model.to(device)

    model.eval()

    # Infer
    pred_list = None

    with torch.no_grad():
        for start_index in tqdm(range(0, num_data, args.batch_size)):
            end_index = min(start_index + args.batch_size, num_data)
            data_batch = data[index_list[start_index:end_index]]
            data_tensor = naive_sparse2tensor(data_batch).to(device)

            #recon_batch = model(data_tensor, calculate_loss=False)
            if model_name == 'vae':
                 recon_batch, _, _ = model(data_tensor)
            elif model_name == 'dae':
                 recon_batch = model(data_tensor)
            else:
                 recon_batch = model(data_tensor, calculate_loss=False)

            recon_batch = recon_batch.cpu().numpy()

            recon_batch[data_batch.nonzero()] = -np.inf

            idx = bn.argpartition(-recon_batch, 500, axis=1)[:, :500]
            out = np.array([])

            for user, item in enumerate(idx):
                user += start_index
                user = id2profile[user]

                infer_out = infer(user, item, dict_user_lv, dict_item_lv, id2show, args.infer_cnt)
                out = np.append(out, infer_out)


            if start_index == 0:
                pred_list = out
            else:
                pred_list = np.append(pred_list, np.array(out))

    user2 = []
    item2 = []

    pred_list = pred_list.reshape(num_data, -1)
    for user_index, pred_item in enumerate(pred_list):
        user2.extend([user_index] * args.infer_cnt)
        item2.extend(pred_item)

    u2 = pd.DataFrame(user2, columns=['user'])
    i2 = pd.DataFrame(item2, columns=['item'])
    all2 = pd.concat([u2, i2], axis=1)

    ans2 = de_numerize(all2, id2profile, id2show)
    ans2.columns = ['user', 'item']
    new_ans2 = ans2.sort_values('user')

    new_ans2.to_csv(os.path.join(args.dataset, 'output.csv'), index=False)
    # print('Output Saved!!')
    return new_ans2

In [14]:
# train_recvae.py

import numpy as np
import torch
from torch import optim
import random
from copy import deepcopy
#from .utils import *
#from .models import *
import time
#from .config import *
#from .dataset import *
import datetime
import json

seed = args.seed
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
import warnings
warnings.filterwarnings(action='ignore')

torch.manual_seed(args.seed)

# 만약 GPU가 사용가능한 환경이라면 GPU를 사용
def train_recvae():
    if torch.cuda.is_available():
        args.cuda = True

    device = torch.device("cuda" if args.cuda else "cpu")
    #device = "cpu"
    print("DEVICE: ", device)
    
    def evaluate(model, data_tr, data_te, metrics, batch_size=500):
        metrics = deepcopy(metrics)
        model.eval()

        e_idxlist = list(range(data_tr.shape[0]))
        e_N = data_tr.shape[0]

        for m in metrics:
            m['score'] = []

        with torch.no_grad():
            for start_idx in range(0, e_N, batch_size):
                end_idx = min(start_idx + batch_size, e_N)
                data = data_tr[e_idxlist[start_idx:end_idx]]
                heldout_data = data_te[e_idxlist[start_idx:end_idx]]

                data_tensor = naive_sparse2tensor(data).to(device)

                ratings_pred = model(data_tensor, calculate_loss=False).cpu().detach().numpy()

                if not (data_tr is data_te):
                    ratings_pred[data.nonzero()] = -np.inf

                for m in metrics:
                    m['score'].append(m['metric'](ratings_pred, heldout_data, k=m['k']))

        for m in metrics:
            m['score'] = np.concatenate(m['score']).mean()

        return [x['score'] for x in metrics]


    def train(model, opts, train_data, batch_size, beta, gamma, dropout_rate):
        model.train()
        np.random.shuffle(idxlist)
        for batch_idx, start_idx in enumerate(range(0, N, batch_size)):
            end_idx = min(start_idx + batch_size, N)

            data = train_data[idxlist[start_idx:end_idx]]
            data = naive_sparse2tensor(data).to(device)

            for optimizer in opts:
                optimizer.zero_grad()

            _, loss = model(data, beta=beta, gamma=gamma, dropout_rate=dropout_rate)
            
            loss.backward()

            for optimizer in opts:
                optimizer.step()

    loader = DataLoader(args.dataset)
    print("loader 완료")
    n_items = loader.load_n_items()
    train_data = loader.load_data('train')
    vad_data_tr, vad_data_te = loader.load_data('validation')
    test_data_tr, test_data_te = loader.load_data('test')

    N = train_data.shape[0]  # 25360
    idxlist = list(range(N))

    model_kwargs = {
        'hidden_dim': args.hidden_dim,
        'latent_dim': args.latent_dim,
        'input_dim': train_data.shape[1]
    }

    metrics = [
        {'metric': ndcg, 'k': 100},
        {'metric': recall, 'k': 10},
        {'metric': recall, 'k': 30},
        {'metric': recall, 'k': 50}
    ]

    best_recall = -np.inf

    model = RecVAE(**model_kwargs).to(device)
    model_best = RecVAE(**model_kwargs).to(device)

    learning_kwargs = {
        'model': model,
        'train_data': train_data,
        'batch_size': args.batch_size,
        'beta': args.beta,
        'gamma': args.gamma
    }

    decoder_params = set(model.decoder.parameters())
    encoder_params = set(model.encoder.parameters())

    optimizer_encoder = optim.Adam(encoder_params, lr=args.lr)
    optimizer_decoder = optim.Adam(decoder_params, lr=args.lr)

    log_dir_name = str(datetime.datetime.now())[0:10] + f'_recvae'
    log_dir = os.path.join(args.log_dir, log_dir_name)

    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)
    # -- wandb initialize with configuration

    for epoch in range(1, args.n_epochs+1):
        if args.not_alternating:
            train(opts=[optimizer_encoder, optimizer_decoder], dropout_rate=0.5, **learning_kwargs)
        else:
            train(opts=[optimizer_encoder], dropout_rate=0.5, **learning_kwargs)
            model.update_prior()
            train(opts=[optimizer_decoder], dropout_rate=0, **learning_kwargs)

        train_scores = evaluate(model, train_data, train_data, metrics)
        valid_scores = evaluate(model, vad_data_tr, vad_data_te, metrics)

        print('-' * 70)
        print(f'epoch {epoch} | previous best recall: {best_recall}')

        print('training set')
        for metric, score in zip(metrics, train_scores):
            print(f"{metric['metric'].__name__}@{metric['k']}:\t{score:.4f}")

        valid_recall = 0

        print('validation set')
        for metric, score in zip(metrics, valid_scores):
            if metric['metric'].__name__ == 'recall' and metric['k'] == 30:
                valid_recall = score
            print(f"{metric['metric'].__name__}@{metric['k']}:\t{score:.4f}")

        if valid_recall > best_recall:
            best_recall = valid_recall
            model_best.load_state_dict(deepcopy(model.state_dict()))
            with open(os.path.join(log_dir, f'best_recvae_' + args.save), 'wb') as f:
                torch.save(model.state_dict(), f)
            print(f"best_model saved!! (save {best_recall})")
        print('-' * 70 + '\n')

    with open(os.path.join(args.save_dir, f'best_recvae_' + args.save), 'wb') as f:
        torch.save(model_best.state_dict(), f)

    final_scores = evaluate(model_best, test_data_tr, test_data_te, metrics)

    for metric, score in zip(metrics, final_scores):
        print(f"{metric['metric'].__name__}@{metric['k']}:\t{score:.4f}")

    pro_dir = os.path.join(args.dataset, 'pro_sg')
    #np.savetxt(pro_dir+'/recvae_result.txt', [final_scores[1]]) # best_recall 저장

    #f = open(f"{pro_dir}/model_score.json", encoding="utf-8")
    #model_score = json.loads(f.read())
    with open(pro_dir + '/model_score.json', 'r', encoding="utf-8") as f:
        model_score = json.load(f)

    model_score['recvae'] = final_scores[1]
    with open(pro_dir + '/model_score.json', 'w', encoding="utf-8") as f:
        json.dump(model_score, f, ensure_ascii=False, indent="\t")

In [15]:
# train_vae.py

import numpy as np
import torch
from torch import optim
import random
from copy import deepcopy
#from .utils import *
#from .models import *
import time
#from .config import *
#from .dataset import *
import datetime
import json
import warnings
warnings.filterwarnings(action='ignore')

seed = args.seed
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)


torch.manual_seed(args.seed)


update_count = 0
def train_vae():
    # 만약 GPU가 사용가능한 환경이라면 GPU를 사용
    if torch.cuda.is_available():
        args.cuda = True
    #args.cuda = False
    device = torch.device("cuda" if args.cuda else "cpu")
    #device = "cpu"
    print("DEVICE: ", device)
    
    def train(model, criterion, optimizer, is_VAE = False):
        # Turn on training mode
        model.train()
        train_loss = 0.0
        start_time = time.time()
        global update_count

        np.random.shuffle(idxlist)
        
        for batch_idx, start_idx in enumerate(range(0, N, args.batch_size)):
            end_idx = min(start_idx + args.batch_size, N)
            
            data = train_data[idxlist[start_idx:end_idx]]
            data = naive_sparse2tensor(data).to(device)
            optimizer.zero_grad()

            if is_VAE:
                if args.total_anneal_steps > 0:
                    anneal = min(args.anneal_cap, 
                                    1. * update_count / args.total_anneal_steps)
                else:
                    anneal = args.anneal_cap

                optimizer.zero_grad()
                recon_batch, mu, logvar = model(data)
                loss = criterion(recon_batch, data, mu, logvar, anneal)
            else:
                recon_batch = model(data)
                loss = criterion(recon_batch, data)

            loss.backward()
            train_loss += loss.item()
            optimizer.step()

            update_count += 1


    def evaluate(model, criterion, data_tr, data_te, is_VAE=False):
        # Turn on evaluation mode
        model.eval()
        total_loss = 0.0
        global update_count
        e_idxlist = list(range(data_tr.shape[0]))
        e_N = data_tr.shape[0]
        n100_list = []
        r10_list = []
        r30_list = []
        
        with torch.no_grad():
            for start_idx in range(0, e_N, args.batch_size):
                end_idx = min(start_idx + args.batch_size, N)
                data = data_tr[e_idxlist[start_idx:end_idx]]
                heldout_data = data_te[e_idxlist[start_idx:end_idx]]

                data_tensor = naive_sparse2tensor(data).to(device)
                if is_VAE :

                    if args.total_anneal_steps > 0:
                        anneal = min(args.anneal_cap, 
                                    1. * update_count / args.total_anneal_steps)
                    else:
                        anneal = args.anneal_cap

                    recon_batch, mu, logvar = model(data_tensor)

                    loss = criterion(recon_batch, data_tensor, mu, logvar, anneal)

                else :
                    recon_batch = model(data_tensor)
                    loss = criterion(recon_batch, data_tensor)

                total_loss += loss.item()

                # Exclude examples from training set
                recon_batch = recon_batch.cpu().numpy()
                recon_batch[data.nonzero()] = -np.inf

                n100 = ndcg(recon_batch, heldout_data, 100)
                r10 = recall(recon_batch, heldout_data, 10)
                r30 = recall(recon_batch, heldout_data, 30)

                n100_list.append(n100)
                r10_list.append(r10)
                r30_list.append(r30)
    
        total_loss /= len(range(0, e_N, args.batch_size))
        n100_list = np.concatenate(n100_list)
        r10_list = np.concatenate(r10_list)
        r30_list = np.concatenate(r30_list)

        return total_loss, np.mean(n100_list), np.mean(r10_list), np.mean(r30_list)

    ###############################################################################
    # Load data
    ###############################################################################
    loader = DataLoader(args.dataset)

    n_items = loader.load_n_items() # 6807
    train_data = loader.load_data('train') # csr_matrix
    vad_data_tr, vad_data_te = loader.load_data('validation')
    test_data_tr, test_data_te = loader.load_data('test')

    N = train_data.shape[0] # 25360
    idxlist = list(range(N))

    ###############################################################################
    # Build the model
    ###############################################################################

    # p_dims = [200, 1200, 3000, n_items] # [200, 600, 6807]
    p_dims = [200, 3000, n_items] # [200, 600, 6807]
    pro_dir2 = os.path.join(args.dataset, 'pro_sg')
    item_tag_emb = pd.read_csv(pro_dir2 + '/item_tag_emb.csv')
    # model = MultiVAE(p_dims).to(device)
    model = MultiVAE(p_dims, tag_emb=item_tag_emb).to(device)


    optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.wd)
    # optimizer = adabound.AdaBound(model.parameters(), lr=1e-3, final_lr=0.1)
    criterion = loss_function_vae

    ###############################################################################
    # Training code
    ###############################################################################

    # best_n100 = -np.inf
    best_r30 = -np.inf
    update_count = 0
    early_stopping = 40
    stopping_cnt = 0

    log_dir_name = str(datetime.datetime.now())[0:10] + '_vae'
    log_dir = os.path.join(args.log_dir, log_dir_name)
    # log_dir = increment_path(log_dir)

    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)

    # -- wandb initialize with configuration
    # wandb.init(config={"model":'Multi-VAE',
    #                 "batch_size": args.batch_size,
    #                 "lr"        : args.lr,
    #                 "epochs"    : args.n_epochs,
    #                 })

    for epoch in range(1, args.n_epochs + 1):
        epoch_start_time = time.time()
        train(model, criterion, optimizer, is_VAE=True)
        
        
        val_loss, n100, r10, r30 = evaluate(model, criterion, vad_data_tr, vad_data_te, is_VAE=True)
        print('-' * 89)
        print('| end of epoch {:3d}/{:3d} | time: {:4.2f}s | valid loss {:4.4f} | '
                'n100 {:5.4f} | r10 {:5.4f} | r30 {:5.4f}'.format(
                    epoch, args.n_epochs, time.time() - epoch_start_time, val_loss,
                    n100, r10, r30))
        print('-' * 89)

        n_iter = epoch * len(range(0, N, args.batch_size))

        # wandb.log({
        #     "vae_val loss": val_loss,
        #     "vae_n100" : n100,
        #     "vae_r10" : r10,
        #     "vae_r20" : r20})

        if r30 > best_r30:
            with open(os.path.join(log_dir, 'best_vae_' + args.save), 'wb') as f:
                torch.save(model.state_dict(), f)
                print(f"Best model saved! r@30 : {r30:.4f}")
            best_r30 = r30
            stopping_cnt = 0
        else:
            print(f'Stopping Count : {stopping_cnt} / {early_stopping}')
            stopping_cnt += 1

        if stopping_cnt > early_stopping:
            print('*****Early Stopping*****')
            break

    # Load the best saved model.
    with open(os.path.join(log_dir, 'best_vae_' + args.save), 'rb') as f:
        model.load_state_dict(torch.load(f))

    with open(os.path.join(args.save_dir, 'best_vae_' + args.save), 'wb') as f:
        torch.save(model.state_dict(), f)

    # Run on test data.
    test_loss, n100, r10, r30 = evaluate(model, criterion, test_data_tr, test_data_te, is_VAE=True)
    print('=' * 89)
    print('| End of training | test loss {:4.4f} | n100 {:4.4f} | r10 {:4.4f} | '
            'r30 {:4.4f}'.format(test_loss, n100, r10, r30))
    print('=' * 89)

    with open(os.path.join(log_dir, "update_count_vae.txt"), "w", encoding='utf-8') as f:
        f.write(str(update_count))

    pro_dir = os.path.join(args.dataset, 'pro_sg')
#    np.savetxt(pro_dir+'/vae_result.txt', [r10]) # best_recall 저장

 #   df_result = pd.read_csv(pro_dir+'/model_score.csv')
 #   x = pd.DataFrame()
 #   x['model_name'] = 'vae'
 #   x['r10'] = r10
 #   df_result = pd.concat([df_result, x], axis=0)
 #   df_result.to_csv(pro_dir+'/model_score.csv', index=False)

    with open(pro_dir +'/model_score.json', 'r', encoding="utf-8") as f:
        model_score = json.load(f)

    model_score['vae'] = r10
    with open(pro_dir + '/model_score.json', 'w', encoding="utf-8") as f:
        json.dump(model_score, f, ensure_ascii=False, indent="\t")

In [16]:
# train_dae.py

import numpy as np
import torch
from torch import optim
import random
from copy import deepcopy
#from .utils import *
#from .models import *
import time
#from .config import *
#from .dataset import *
import datetime
import json
import warnings
warnings.filterwarnings('ignore')

seed = args.seed
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)


torch.manual_seed(args.seed)

update_count = 0
def train_dae():
    # 만약 GPU가 사용가능한 환경이라면 GPU를 사용
    if torch.cuda.is_available():
        args.cuda = True

    device = torch.device("cuda" if args.cuda else "cpu")
    #device = "cpu"
    print("DEVICE: ", device)

    def train(model, criterion, optimizer, is_VAE=False):
        # Turn on training mode
        model.train()
        train_loss = 0.0
        global update_count

        np.random.shuffle(idxlist)

        for batch_idx, start_idx in enumerate(range(0, N, args.batch_size)):
            end_idx = min(start_idx + args.batch_size, N)

            data = train_data[idxlist[start_idx:end_idx]]
            data = naive_sparse2tensor(data).to(device)

            optimizer.zero_grad()

            if is_VAE:
                if args.total_anneal_steps > 0:
                    anneal = min(args.anneal_cap,
                                1. * update_count / args.total_anneal_steps)
                else:
                    anneal = args.anneal_cap

                optimizer.zero_grad()
                recon_batch, mu, logvar = model(data)
                loss = criterion(recon_batch, data, mu, logvar, anneal)
            else:
                recon_batch = model(data)
                loss = criterion(recon_batch, data)

            loss.backward()
            train_loss += loss.item()
            optimizer.step()

            update_count += 1

    def evaluate(model, criterion, data_tr, data_te, is_VAE=False):
        # Turn on evaluation mode
        model.eval()
        total_loss = 0.0
        global update_count
        e_idxlist = list(range(data_tr.shape[0]))
        e_N = data_tr.shape[0]
        n100_list = []
        r10_list = []
        r30_list = []

        with torch.no_grad():
            for start_idx in range(0, e_N, args.batch_size):
                end_idx = min(start_idx + args.batch_size, N)
                data = data_tr[e_idxlist[start_idx:end_idx]]
                heldout_data = data_te[e_idxlist[start_idx:end_idx]]

                data_tensor = naive_sparse2tensor(data).to(device)
                if is_VAE:

                    if args.total_anneal_steps > 0:
                        anneal = min(args.anneal_cap,
                                    1. * update_count / args.total_anneal_steps)
                    else:
                        anneal = args.anneal_cap

                    recon_batch, mu, logvar = model(data_tensor)

                    loss = criterion(recon_batch, data_tensor, mu, logvar, anneal)

                else:
                    recon_batch = model(data_tensor)
                    loss = criterion(recon_batch, data_tensor)

                total_loss += loss.item()

                # Exclude examples from training set
                recon_batch = recon_batch.cpu().numpy()
                recon_batch[data.nonzero()] = -np.inf

                n100 = ndcg(recon_batch, heldout_data, 100)
                r10 = recall(recon_batch, heldout_data, 10)
                r30 = recall(recon_batch, heldout_data, 30)

                n100_list.append(n100)
                r10_list.append(r10)
                r30_list.append(r30)

        total_loss /= len(range(0, e_N, args.batch_size))
        n100_list = np.concatenate(n100_list)
        r10_list = np.concatenate(r10_list)
        r30_list = np.concatenate(r30_list)

        return total_loss, np.mean(n100_list), np.mean(r10_list), np.mean(r30_list)


    ###############################################################################
    # Load data
    ###############################################################################
    loader = DataLoader(args.dataset)

    n_items = loader.load_n_items()  # 6807
    train_data = loader.load_data('train')  # csr_matrix
    vad_data_tr, vad_data_te = loader.load_data('validation')
    test_data_tr, test_data_te = loader.load_data('test')

    N = train_data.shape[0]  # 25360
    idxlist = list(range(N))

    ###############################################################################
    # Build the model
    ###############################################################################

    p_dims = [200, 3000, n_items] 
    pro_dir2 = os.path.join(args.dataset, 'pro_sg')
    item_tag_emb = pd.read_csv(pro_dir2 + '/item_tag_emb.csv')
    model = MultiDAE(p_dims, tag_emb=item_tag_emb).to(device)

    optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.wd)
    # optimizer = adabound.AdaBound(model.parameters(), lr=1e-3, final_lr=0.1)
    criterion = loss_function_dae

    ###############################################################################
    # Training code
    ###############################################################################

    # best_n100 = -np.inf
    best_r30 = -np.inf
    update_count = 0
    early_stopping = 40
    stopping_cnt = 0

    log_dir_name = str(datetime.datetime.now())[0:10] + '_dae'
    log_dir = os.path.join(args.log_dir, log_dir_name)


    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)

    for epoch in range(1, args.n_epochs + 1):
        epoch_start_time = time.time()
        train(model, criterion, optimizer, is_VAE=False)

        val_loss, n100, r10, r30 = evaluate(model, criterion, vad_data_tr, vad_data_te, is_VAE=False)
        print('-' * 89)
        print('| end of epoch {:3d}/{:3d} | time: {:4.2f}s | valid loss {:4.4f} | '
            'n100 {:5.4f} | r10 {:5.4f} | r30 {:5.4f}'.format(
            epoch, args.n_epochs, time.time() - epoch_start_time, val_loss,
            n100, r10, r30))
        print('-' * 89)

        n_iter = epoch * len(range(0, N, args.batch_size))

        if r30 > best_r30:
            with open(os.path.join(log_dir, 'best_dae_' + args.save), 'wb') as f:
                torch.save(model.state_dict(), f)
                print(f"Best model saved! r@30 : {r30:.4f}")
            best_r30 = r30
            stopping_cnt = 0
        else:
            print(f'Stopping Count : {stopping_cnt} / {early_stopping}')
            stopping_cnt += 1

        if stopping_cnt > early_stopping:
            print('*****Early Stopping*****')
            break

    # Load the best saved model.
    with open(os.path.join(log_dir, 'best_dae_' + args.save), 'rb') as f:
        model.load_state_dict(torch.load(f))

    with open(os.path.join(args.save_dir, 'best_dae_' + args.save), 'wb') as f:
        torch.save(model.state_dict(), f)

    # Run on test data.
    test_loss, n100, r10, r30 = evaluate(model, criterion, test_data_tr, test_data_te, is_VAE=False)
    print('=' * 89)
    print('| End of training | test loss {:4.4f} | n100 {:4.4f} | r10 {:4.4f} | '
        'r30 {:4.4f}'.format(test_loss, n100, r10, r30))
    print('=' * 89)

    with open(os.path.join(log_dir, "update_count_dae.txt"), "w", encoding='utf-8') as f:
        f.write(str(update_count))

    
    pro_dir = os.path.join(args.dataset, 'pro_sg')
#    np.savetxt(pro_dir+'/dae_result.txt', [r10]) # best_recall 저장


#    df_result = pd.read_csv(pro_dir+'/model_score.csv')
#    x = pd.DataFrame()
#    x['model_name'] = 'dae'
#    x['r10'] = r10
#    df_result = pd.concat([df_result, x], axis=0)
#    df_result.to_csv(pro_dir+'/model_score.csv', index=False)

    with open(pro_dir + '/model_score.json', 'r', encoding="utf-8") as f:
        model_score = json.load(f)

    model_score['dae'] = r10
    with open(pro_dir + '/model_score.json', 'w', encoding="utf-8") as f:
        json.dump(model_score, f, ensure_ascii=False, indent="\t")

## main.py

In [17]:
#from .preprocessing import preprocessing_all
#from .inference import infer_all
#from .train_recvae import train_recvae
#from .train_vae import train_vae
#from .train_dae import train_dae
import pandas as pd
import numpy as np
#from .utils import *
#from .config import *
import json

In [ ]:
#df_users = pd.read_csv('/gdrive/My Drive/Colab Notebooks/SSAFY/users.csv')  # 유저 데이터
#df_problems = pd.read_csv('/gdrive/My Drive/Colab Notebooks/SSAFY/problems.csv')  # 문제 데이터
#df_problems_solved = pd.read_csv('/gdrive/My Drive/Colab Notebooks/SSAFY/user_solved_problems_fixed.csv')  # 유저별 푼 문제 데이터
#df_records_solved = pd.read_csv('/gdrive/My Drive/Colab Notebooks/SSAFY/user_lately_solved_problems2.csv')  # 유저별 최근 60개의 '맞혔습니다' 데이터

In [18]:
def make_train_data(db):
    #df_problems_solved = pd.read_sql('SELECT * FROM problems_solved', db)
    df_problems_solved = pd.read_csv('/gdrive/My Drive/Colab Notebooks/SSAFY/user_solved_problems_fixed.csv')  # 유저별 푼 문제 데이터
    df_problems_solved['problems'] = df_problems_solved['problems'].str.split(',')
    df_problems_solved = df_problems_solved.replace(['', 'null'], [np.nan, np.nan])
    df_problems_solved = df_problems_solved.explode('problems').dropna().reset_index(drop=True)
    df_problems_solved = df_problems_solved.replace(['', 'null'], [np.nan, np.nan])
    df_problems_solved = df_problems_solved.dropna()
    df_problems_solved = df_problems_solved.drop(columns=['id'])
    df_problems_solved = df_problems_solved.astype({'handle':'str', 'problems':'int'})
    df_problems_solved.columns = ['user', 'item']

    raw_data, user_activity, item_popularity = filter_triplets(df_problems_solved, 5, 10)
    #---------------------------
    # 여기에 tag == None인 아이템은 제외
    #df_problems = pd.read_sql('select * from problems', db)
    df_problems = pd.read_csv('/gdrive/My Drive/Colab Notebooks/SSAFY/problems.csv')  # 문제 데이터
    df_problems.drop(df_problems[df_problems.average_tries == 7340].index, axis=0, inplace=True)
    # level 0에 해당하는 문제 제거
    df_problems = df_problems[df_problems.level != 0]
    # not_solvable == False만
    df_problems = df_problems[df_problems.is_solvable == True]
    # tag가 nan인 문제 제거
    df_problems = df_problems[~df_problems.tags.isnull()]

    gudegi = [24900, 24901, 24902, 24903, 24904, 24905, 24906, 24907, 24908, 24909, 24910, 24911, 21292, 21293, 21294, 21295, 21296, 21297, 21298, 21299, 18821, 18822, 18823, 18824, 18825, 18826, 18827, 18828, 18829, 18830, 18831, 18832, 18833, 18834, 18835, 18836, 17106, 17107, 17108, 17109, 17110, 17111, 17112, 17113, 17114, 17115, 17116, 17117, 17118, 17119, 17120, 15629, 15630, 15631, 15632, 15633, 15634, 15635, 15636, 15637, 15638, 15639, 15640, 15641, 15642, 15643]
    df_problems = df_problems[~df_problems['problemId'].isin(gudegi)].reset_index(drop=True)

    raw_data = raw_data[raw_data['item'].isin(df_problems['problemId'].values)].reset_index(drop=True)
    #---------------------------

    return raw_data, df_problems


def general_problem_preprocessing(db):
    raw_data, df_problems = make_train_data(db)

    print("Preprocessing Start!!")
    preprocessing_all(raw_data, df_problems, db)


def recvae_train():
    print("Train Start!!")
    train_recvae()

def vae_train():
    print("Train Start!!")
    train_vae()

def dae_train():
    print("Train Start!!")
    train_dae()

def general_pb_infer(db):
    raw_data, df_problems = make_train_data(db)

    print("Inference Start!!")
    output = infer_all(raw_data, df_problems, db)
    output.item = output.item.astype(str)
    output = output.groupby('user')['item'].apply(lambda x: "%s" % ','.join(x))
    output = pd.DataFrame(output)
    output = output.reset_index()

    return output

#if __name__ == '__main__':
#   df_problems_solved = 데이터 읽기
#   raw_data = make_train_data(df_problems_solved)
#   print(from_pre_to_infer(raw_data))